In [6]:
import requests, json, textwrap, pprint
url = "http://127.0.0.1:8000/chat"
payload = {
    "user_query": "¿Qué significa tener hemoglobina baja?",
    "patient_id": "123456"
}
print(requests.post(url, json=payload).json())

{'answer': ''}


In [9]:
from langchain_chroma import Chroma
db = Chroma(persist_directory="RAG/chroma")
print("Documentos en el índice:", db._collection.count())

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Documentos en el índice: 7596


In [10]:
bot = RAG_chatbot()
bot.populate_chroma()   # ejecuta y genera embeddings

NameError: name 'RAG_chatbot' is not defined

In [12]:
from services.fhir_client import FHIRClient
f = FHIRClient()
print(f.get_patient_by_id("123456"))
print(f.get_observations("123456"))
print(f.get_medications("123456"))

HTTPStatusError: Client error '401 Unauthorized' for url 'https://cancer-care-fhir-api.onrender.com/fhir/Patient?_id=123456'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401

In [3]:
import requests

url = "https://cancer-care-fhir-api.onrender.com/fhir/Patient?_count=1"
headers = {"Accept": "application/fhir+json"}

resp = requests.get(url, headers=headers, timeout=20)

print("Código:", resp.status_code)
print("JSON:", resp.json())

Código: 401
JSON: {'error': 'Falta header de autorización o es inválido'}


In [4]:
from services.fhir_client import FHIRClient
import pprint, textwrap

fhir = FHIRClient()           # ← usa las credenciales de tu clase

# 1) Bundle con 1 paciente
bundle = fhir.get_patients(count=1)
pprint.pprint(bundle)

# 2) Extrae el primer ID real
patient_id = bundle["entry"][0]["resource"]["id"]
print("\n✅ patient_id =", patient_id)

# 3) Pide el recurso Patient completo
patient = fhir.get_patient_by_id(patient_id)
print("\nNombre:",
      patient["name"][0]["given"][0],
      patient["name"][0]["family"])

# 4) Observaciones
labs = fhir.get_observations(patient_id)
print("\nObservations entries:", len(labs.get("entry", [])))

# 5) Medicaciones
meds = fhir.get_medications(patient_id)
print("Medication entries :", len(meds.get("entry", [])))


{'entry': [{'resource': {'birthDate': '1990-04-13',
                         'gender': 'male',
                         'id': 'paciente-duplicado',
                         'identifier': [{'system': 'http://hospital.saludinteligente.ejemplo/pacientes',
                                         'value': 'MRN279933'}],
                         'name': [{'family': 'López', 'given': ['Ana']}],
                         'resourceType': 'Patient'}},
           {'resource': {'birthDate': '1966-10-15',
                         'gender': 'female',
                         'id': 'paciente-1',
                         'identifier': [{'system': 'http://hospital.saludinteligente.ejemplo/pacientes',
                                         'value': 'MRN856268'}],
                         'name': [{'family': 'González', 'given': ['José']}],
                         'resourceType': 'Patient'}},
           {'resource': {'birthDate': '1955-05-17',
                         'gender': 'female',
             

In [17]:
from services.fhir_client import FHIRClient
f = FHIRClient()
patient = f.get_patient_by_id("paciente-1")   # o el id real que obtuviste
print(patient["name"][0]["given"][0])

María


In [6]:
payload = {
    "user_query": "¿Qué significa tener hemoglobina baja?",
    "patient_id": "paciente-1"    # o el id que acabas de probar
}
print(requests.post("http://127.0.0.1:8000/chat", json=payload).json())


{'answer': ''}


In [20]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="hf.co/sathvik123/llama3-ChatDoc")
print(llm.invoke("Explícame brevemente qué es la anemia."))

¡Hola! La anemia es un trastorno médico que se caracteriza por una disminución del conteo de glóbulos rojos (eritrocitos) en sangre. Esto puede ocasionar un defectuoso transporte de oxígeno hacia las células, lo que puede provocar sintomatología como fatiga, debilidad, palidez y dolor de cabeza. Hay diferentes tipos de anemia, algunos causados por deficiencias nutricionales (anemia ferropénica) o enfermedades crónicas (anemia renal). ¡Espero haberte ayudado!


In [9]:
import requests

url = "http://127.0.0.1:8000/chat"
payload = {
    "user_query": "¿Qué significa tener hemoglobina baja?",
    "patient_id": "paciente-1"
}

response = requests.post(url, json=payload)
print("Respuesta del modelo:")
print(response.json())

Respuesta del modelo:
{'answer': ''}


In [2]:
from services.fhir_client import FHIRClient
fhir = FHIRClient()

bundle = fhir.get_patients(count=1)
patient_id = bundle["entry"][0]["resource"]["id"]
print("✅ patient_id:", patient_id)

# Paciente
patient = fhir.get_patient_by_id(patient_id)
print("👤 Nombre:", patient["name"][0]["given"][0])

# Observaciones
labs = fhir.get_observations(patient_id)
print("🧪 Labs entries:", len(labs.get("entry", [])))

# Medicaciones (ya con fallback)
meds = fhir.get_medications(patient_id)
print("💊 Meds entries:", len(meds.get("entry", [])))

✅ patient_id: paciente-0
👤 Nombre: María
🧪 Labs entries: 8
💊 Meds entries: 0


In [9]:
from services.hl7_client import HL7Client

hl7 = HL7Client()

# Ver los mensajes actuales
msgs = hl7.get_messages()
print("📨 Mensajes HL7 recibidos:", msgs["total"])
for msg in msgs["messages"]:
    print(msg["message"])

# Ver estadísticas
stats = hl7.get_stats()
print("📊 Stats:", stats)

# Ver detalles del stream
info = hl7.get_stream_info()
print("ℹ️ Info del Stream:", info)

KeyError: 'total'

In [2]:
from RAG.rag import RAG_chatbot
bot = RAG_chatbot()
print(
    bot.app.invoke(
        {
            "question": "¿Es preocupante un valor de WBC de 7.5?",
            "patient_id": "paciente-1"
        }
    )["llm_output"]
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



=== DEBUG generate_answer ===
Pregunta: ¿Es preocupante un valor de WBC de 7.5?
patient_context → 'Paciente Carmen (ID paciente-1). Hemoglobina: 14.0 g/dL. Medicamento actual: (sin medicación registrada).'
docs recuperados → 4
⤷ primer doc: ing. Ann Cardiothorac Surg. 2013;2:427–430.
 7 . Gaudino M, Benedetto U, Fremes S, et al. Radial-artery or saphenous-
ve …


🧪 PROMPT COMPLETO QUE RECIBE EL MODELO:

System: Eres un profesional médico.
                Primero revisa la ficha clínica del paciente.
                Luego usa los documentos recuperados para justificar tu respuesta.
                === Información del paciente ===
                Paciente Carmen (ID paciente-1). Hemoglobina: 14.0 g/dL. Medicamento actual: (sin medicación registrada).
                === Documentos de referencia ===
                ing. Ann Cardiothorac Surg. 2013;2:427–430.
 7 . Gaudino M, Benedetto U, Fremes S, et al. Radial-artery or saphenous-
vein grafts in coronary-artery bypass surgery. N Engl J M

KeyError: "Input to ChatPromptTemplate is missing variables {'hl7_context'}.  Expected: ['chat_history', 'context', 'hl7_context', 'input', 'patient_context'] Received: ['input', 'patient_context', 'chat_history', 'context']\nNote: if you intended {hl7_context} to be part of the string and not a variable, please escape it with double curly braces like: '{{hl7_context}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [2]:
from RAG.rag import RAG_chatbot
bot = RAG_chatbot()
print(
    bot.app.invoke(
        {"question": "¿Es preocupante un valor de WBC de 7.5?",
         "patient_id": "paciente-1"}
    )["llm_output"]
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



=== DEBUG generate_answer ===
Pregunta: ¿Es preocupante un valor de WBC de 7.5?
patient_context → 'Paciente Ana (ID paciente-1). Hemoglobina: 12.0 g/dL. Medicamento actual: (sin medicación registrada).'
docs recuperados → 4
⤷ primer doc: ing. Ann Cardiothorac Surg. 2013;2:427–430.
 7 . Gaudino M, Benedetto U, Fremes S, et al. Radial-artery or saphenous-
ve …


🧪 PROMPT COMPLETO QUE RECIBE EL MODELO:

System: Eres un profesional médico.
                Primero revisa la ficha clínica del paciente.
                Luego usa los documentos recuperados para justificar tu respuesta.
                === Información del paciente ===
                Paciente Ana (ID paciente-1). Hemoglobina: 12.0 g/dL. Medicamento actual: (sin medicación registrada).
                === Documentos de referencia ===
                ing. Ann Cardiothorac Surg. 2013;2:427–430.
 7 . Gaudino M, Benedetto U, Fremes S, et al. Radial-artery or saphenous-
vein grafts in coronary-artery bypass surgery. N Engl J Med. 20

In [1]:
from RAG.rag import RAG_chatbot

bot = RAG_chatbot()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [2]:
user_query = (
    "Desde ayer tengo fatiga extrema, náuseas y entumecimiento en las manos. "
    "¿Debo preocuparme? ¿Qué exámenes necesito?"
)

patient_id = "paciente-1"  

In [3]:
response = bot.app.invoke(
    {
        "question": user_query,
        "patient_id": patient_id
    }
)

print("🧠 RESPUESTA FINAL DEL LLM:\n")
print(response["llm_output"])

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



=== DEBUG generate_answer ===
Pregunta: Desde ayer tengo fatiga extrema, náuseas y entumecimiento en las manos. ¿Debo preocuparme? ¿Qué exámenes necesito?
patient_context → 'Error al obtener datos FHIR para paciente-1: The read operation timed out'
docs recuperados → 4
⤷ primer doc: criterion, especially because other entities dem-
onstrate comparable response (eg, esophageal 
spasm).1,5 Associated sy …


🧪 PROMPT COMPLETO QUE RECIBE EL MODELO:

System: Eres un profesional médico.
                Primero revisa la ficha clínica del paciente.
                Luego usa los documentos recuperados para justificar tu respuesta.
                === Información del paciente ===
                Error al obtener datos FHIR para paciente-1: The read operation timed out
                === Documentos de referencia ===
                criterion, especially because other entities dem-
onstrate comparable response (eg, esophageal 
spasm).1,5 Associated symptoms such as short-
ness of breath, nausea

In [1]:
from RAG.rag import RAG_chatbot

bot = RAG_chatbot()

resp = bot.app.invoke({
    "question": "Mi paciente oncológico tiene fatiga crónica y está tomando tamoxifeno. ¿Hay alertas?",
    "patient_id": "paciente-1"
})

print(resp["llm_output"])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



=== DEBUG generate_answer ===
Pregunta: Mi paciente oncológico tiene fatiga crónica y está tomando tamoxifeno. ¿Hay alertas?
patient_context → 'Error al obtener datos FHIR para paciente-1: RetryError[<Future at 0x22a33207b10 state=finished raised TypeError>]'
docs recuperados → 4
⤷ primer doc: the disease. Reducing the prevalence and incidence of a preventable disease first begins with discovering the cause of t …


🧪 PROMPT COMPLETO QUE RECIBE EL MODELO:

System: Eres un profesional médico.
                Primero revisa la ficha clínica del paciente.
                Luego usa los documentos recuperados para justificar tu respuesta.
                === Información del paciente ===
                Error al obtener datos FHIR para paciente-1: RetryError[<Future at 0x22a33207b10 state=finished raised TypeError>]
                === Documentos de referencia ===
                the disease. Reducing the prevalence and incidence of a preventable disease first begins with discovering the ca